# warp-md Agent Framework Quickstart

This notebook demonstrates how to integrate warp-md with popular agent frameworks.

## Prerequisites

1. Install warp-md: `pip install warp-md`
2. Install your chosen framework (LangChain, CrewAI, etc.)
3. Set API key for your LLM provider

## 1. Direct Python API (No Framework)

For simple use cases, you don't need an agent framework at all.

In [ ]:
import json
import subprocess
from pathlib import Path

def run_warp_md_analysis(topology: str, trajectory: str, analyses: list) -> dict:
    """Run warp-md analysis and return result envelope."""
    request = {
        "version": "warp-md.agent.v1",
        "system": topology,
        "trajectory": trajectory,
        "analyses": analyses,
    }
    
    config_path = Path("_temp_request.json")
    with open(config_path, "w") as f:
        json.dump(request, f)
    
    result = subprocess.run(
        ["warp-md", "run", str(config_path)],
        capture_output=True,
        text=True,
    )
    
    config_path.unlink()
    return json.loads(result.stdout)

# Example: Calculate Rg
# result = run_warp_md_analysis(
#     "protein.pdb",
#     "traj.xtc",
#     [{"name": "rg", "selection": "protein"}]
# )
# print(result)

## 2. LangChain Integration

Install: `pip install langchain langchain-core langchain-openai`

In [ ]:
# Uncomment to install
# %pip install langchain langchain-core langchain-openai

import os
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_openai import ChatOpenAI
from langchain import hub

# Set API key
# os.environ["OPENAI_API_KEY"] = "your-key-here"

# Import the tool
import sys
sys.path.insert(0, "../agents/langchain")
from warp_md_tool import create_warp_md_tool

# Create tool
warp_tool = create_warp_md_tool()

# Create agent
llm = ChatOpenAI(model="gpt-4o", temperature=0)
prompt = hub.pull("hwchase17/openai-tools")
agent = create_tool_calling_agent(llm, [warp_tool], prompt)
executor = AgentExecutor(agent=agent, tools=[warp_tool], verbose=True)

# Run analysis
# result = executor.invoke({
#     "input": "Calculate radius of gyration for the protein in protein.pdb and traj.xtc"
# })

## 3. CrewAI Integration

Install: `pip install crewai crewai-tools`

In [ ]:
# Uncomment to install
# %pip install crewai crewai-tools

from crewai import Agent, Task, Crew

import sys
sys.path.insert(0, "../agents/crewai")
from warp_md_tool import WarpMDAnalysisTool

# Create analyst agent
analyst = Agent(
    role="MD Analyst",
    goal="Execute accurate trajectory analyses",
    backstory="Expert computational chemist with 15 years experience",
    tools=[WarpMDAnalysisTool()],
    verbose=True,
)

# Define task
task = Task(
    description="Calculate Rg, RMSD, and end-to-end distance for the protein",
    expected_output="Result file paths",
    agent=analyst,
)

# Execute
# crew = Crew(agents=[analyst], tasks=[task])
# result = crew.kickoff()

## 4. AutoGen Integration

Install: `pip install pyautogen`

In [ ]:
# Uncomment to install
# %pip install pyautogen

from autogen import AssistantAgent, UserProxyAgent

import sys
sys.path.insert(0, "../agents/autogen")
from warp_md_tool import create_warp_md_tool_decorator

# Create pre-configured tool
md_tool = create_warp_md_tool_decorator(
    topology="protein.pdb",
    trajectory="traj.xtc",
    output_dir="results"
)

# Create agents
user_proxy = UserProxyAgent(
    name="user",
    human_input_mode="NEVER",
    function_map={"warp_md_analysis": md_tool},
)

analyst = AssistantAgent(
    name="analyst",
    llm_config={"config_list": [{"model": "gpt-4o"}]},
)

# Start conversation
# user_proxy.initiate_chat(
#     analyst,
#     message="Analyze the protein structure stability"
# )

## Common Analysis Patterns

### Structural Analysis
```json
[{"name": "rg", "selection": "protein"}]
[{"name": "rmsd", "selection": "protein", "align": true}]
[{"name": "end_to_end", "selection": "protein"}]
```

### Solvent Analysis
```json
[{
    "name": "rdf",
    "sel_a": "resname SOL and name OW",
    "sel_b": "resname SOL and name OW",
    "bins": 200,
    "r_max": 10.0
}]
```

### Transport Analysis
```json
[{
    "name": "conductivity",
    "selection": "resname NA or resname CL",
    "charges": [1.0, -1.0],
    "temperature": 300.0
}] 
```